# Проект по SQL

**Цель исследования: проанализировать базу данных крупного сервиса для чтения книг по подписке.**

Задачи исследования:

 - Исследуйте таблицы — выведите первые строки. 
 - Посчитайте, сколько книг вышло после 1 января 2000 года;
 - Для каждой книги посчитайте количество обзоров и среднюю оценку;
 - Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;
 - Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
 - Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine

In [2]:
# устанавливаем параметры
db_config = {'user': '...', # имя пользователя
'pwd': '...', # пароль
'host': '...',
'port': 6432, # порт подключения
'db': '.'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

In [4]:
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

## Исследуйте таблицы — выведите первые строки.

### Таблица books (книги)

In [5]:
# чтобы выполнить SQL-запрос, используем Pandas
query = '''SELECT * FROM books LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


### Таблица authors (авторы)

In [110]:
# чтобы выполнить SQL-запрос, используем Pandas
query = '''SELECT * FROM authors LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


### Таблица publishers (издатели)

In [7]:
# чтобы выполнить SQL-запрос, используем Pandas
query = '''SELECT * FROM publishers LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


### Таблица ratings (рейтинги)

In [8]:
# чтобы выполнить SQL-запрос, используем Pandas
query = '''SELECT * FROM ratings LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


### Таблица reviews (обзоры)

In [9]:
# чтобы выполнить SQL-запрос, используем Pandas
query = '''SELECT * FROM reviews LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


## Посчитайте, сколько книг вышло после 1 января 2000 года

In [12]:
# чтобы выполнить SQL-запрос, используем Pandas
query = '''
SELECT COUNT(*) FROM books
WHERE publication_date > '2000-01-01'
'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,count
0,819


*Вывод по п.2: после 1 января 2000 года вышло 819 книг.*

## Для каждой книги посчитайте количество обзоров и среднюю оценку

In [108]:
# чтобы выполнить SQL-запрос, используем Pandas
query = '''
WITH ob AS (SELECT book_id, COUNT(review_id) AS count_reviews
            FROM reviews
            GROUP BY book_id),
     oc AS (SELECT book_id, ROUND(AVG(rating), 2) AS avg_rating
            FROM ratings
            GROUP BY book_id)

SELECT b.book_id, b.title, count_reviews, oc.avg_rating
FROM books AS b LEFT OUTER JOIN ob ON b.book_id=ob.book_id
                LEFT OUTER JOIN oc ON b.book_id=oc.book_id

'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,title,count_reviews,avg_rating
0,652,The Body in the Library (Miss Marple #3),2.0,4.50
1,273,Galápagos,2.0,4.50
2,51,A Tree Grows in Brooklyn,5.0,4.25
3,951,Undaunted Courage: The Pioneering First Missio...,2.0,4.00
4,839,The Prophet,4.0,4.29
...,...,...,...,...
995,672,The Cat in the Hat and Other Dr. Seuss Favorites,NaN,5.00
996,83,Anne Rice's The Vampire Lestat: A Graphic Novel,NaN,3.67
997,221,Essential Tales and Poems,NaN,4.00
998,387,Leonardo's Notebooks,NaN,4.00


*Вывод по п.3: по таблице видно Название книги (title), Количество обзоров у книги (count_reviews), Средний рейтинг книги (avg_rating). Не у всех книг есть обзоры.*

## Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры

In [33]:
# чтобы выполнить SQL-запрос, используем Pandas
query = '''
SELECT publisher
FROM publishers
WHERE publisher_id IN (
                        SELECT publisher_id
                        FROM books
                        WHERE num_pages > 50
                        GROUP BY publisher_id
                        ORDER BY COUNT(book_id) DESC
                        LIMIT 1
                        )

'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,publisher
0,Penguin Books


*Вывод по п.4: издательство "Penguin Books" выпустило наибольшее число книг толще 50 страниц.*

## Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками

In [88]:
 # чтобы выполнить SQL-запрос, используем Pandas
query = '''
SELECT author
FROM authors
WHERE author_id IN (
                    SELECT b.author_id
                    FROM (
                        SELECT book_id, 
                            AVG(rating) AS avg_rating, 
                            COUNT(rating) count_rating
                        FROM ratings
                        GROUP BY book_id) AS av LEFT OUTER JOIN books AS b ON av.book_id=b.book_id 
                        WHERE count_rating >50
                        GROUP BY b.author_id
                        ORDER BY AVG(avg_rating) DESC
                        LIMIT 1
                        )
 '''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con) 

,author
0,J.K. Rowling/Mary GrandPré


*Вывод по п.5: автор "J.K. Rowling/Mary GrandPré" имеет самый высокий средний рейтинг написанных им книг среди книг с 50 и более оценками.*

## Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

In [107]:
 # чтобы выполнить SQL-запрос, используем Pandas
query = '''
WITH ra AS (SELECT username, COUNT(rating) AS count_ratings
            FROM ratings
            GROUP BY username),
     re AS (SELECT username, COUNT(text) AS count_reviews
            FROM reviews
            GROUP BY username)

SELECT AVG(re.count_reviews) AS avg_count_reviews
FROM ra LEFT OUTER JOIN re ON ra.username=re.username
WHERE ra.count_ratings > 48
 '''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con) 

,avg_count_reviews
0,24.0


*Вывод по п.6: Среднее количество обзоров от пользователей, которые поставили больше 48 оценок равно 24.*

## Общий вывод

*Все задачи исследования выполнены.*